# Connect Four

In [32]:
import math
import random

ROWS = 6
COLUMNS = 7
WINDOW_LENGTH = 4
EMPTY = 0
PLAYER_PIECE = 1  # 'O'
AI_PIECE = 2      # 'X'

def create_board():
    board = [[0 for _ in range(COLUMNS)] for _ in range(ROWS)]
    return board

def get_piece_symbol(piece):
    if piece == PLAYER_PIECE:
        return 'O'
    elif piece == AI_PIECE:
        return 'X'
    else:
        return '.'

def print_board(board, winning_positions=None):
    print("  ".join([str(i) for i in range(COLUMNS)]))
    for r, row in enumerate(board):
        symbols = []
        for c, cell in enumerate(row):
            symbol = get_piece_symbol(cell)
            if winning_positions and (r, c) in winning_positions:
                symbol = '\033[1m\033[32m' + symbol + '\033[0m'  # Make symbol bold
            symbols.append(symbol)
        print("  ".join(symbols))
    print()

def is_valid_location(board, col):
    return board[0][col] == EMPTY

def get_next_open_row(board, col):
    for r in range(ROWS-1, -1, -1):
        if board[r][col] == EMPTY:
            return r

def drop_piece(board, row, col, piece):
    board[row][col] = piece

def winning_move(board, piece):
    positions = []
    # Check horizontal locations
    for c in range(COLUMNS - 3):
        for r in range(ROWS):
            if (board[r][c] == piece and board[r][c+1] == piece and
                board[r][c+2] == piece and board[r][c+3] == piece):
                positions = [(r, c+i) for i in range(4)]
                return positions
    # Check vertical locations
    for c in range(COLUMNS):
        for r in range(ROWS - 3):
            if (board[r][c] == piece and board[r+1][c] == piece and
                board[r+2][c] == piece and board[r+3][c] == piece):
                positions = [(r+i, c) for i in range(4)]
                return positions
    # Check positively sloped diagonals
    for c in range(COLUMNS - 3):
        for r in range(ROWS - 3):
            if (board[r][c] == piece and board[r+1][c+1] == piece and
                board[r+2][c+2] == piece and board[r+3][c+3] == piece):
                positions = [(r+i, c+i) for i in range(4)]
                return positions
    # Check negatively sloped diagonals
    for c in range(COLUMNS - 3):
        for r in range(3, ROWS):
            if (board[r][c] == piece and board[r-1][c+1] == piece and
                board[r-2][c+2] == piece and board[r-3][c+3] == piece):
                positions = [(r-i, c+i) for i in range(4)]
                return positions
    return []

def evaluate_window(window, piece):
    score = 0
    opp_piece = PLAYER_PIECE if piece == AI_PIECE else AI_PIECE

    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(EMPTY) == 1:
        score += 5
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        score += 2

    if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
        score -= 4

    return score

def score_position(board, piece):
    score = 0
    # Score center column
    center_array = [board[r][COLUMNS//2] for r in range(ROWS)]
    center_count = center_array.count(piece)
    score += center_count * 3

    # Score Horizontal
    for r in range(ROWS):
        row_array = [board[r][c] for c in range(COLUMNS)]
        for c in range(COLUMNS - 3):
            window = row_array[c:c+WINDOW_LENGTH]
            score += evaluate_window(window, piece)

    # Score Vertical
    for c in range(COLUMNS):
        col_array = [board[r][c] for r in range(ROWS)]
        for r in range(ROWS - 3):
            window = col_array[r:r+WINDOW_LENGTH]
            score += evaluate_window(window, piece)

    # Score positive sloped diagonal
    for r in range(ROWS - 3):
        for c in range(COLUMNS - 3):
            window = [board[r+i][c+i] for i in range(WINDOW_LENGTH)]
            score += evaluate_window(window, piece)

    # Score negative sloped diagonal
    for r in range(3, ROWS):
        for c in range(COLUMNS - 3):
            window = [board[r-i][c+i] for i in range(WINDOW_LENGTH)]
            score += evaluate_window(window, piece)

    return score

def is_terminal_node(board):
    return bool(winning_move(board, PLAYER_PIECE)) or bool(winning_move(board, AI_PIECE)) or len(get_valid_locations(board)) == 0

def minimax(board, depth, alpha, beta, maximizingPlayer):
    valid_locations = get_valid_locations(board)
    is_terminal = is_terminal_node(board)
    if depth == 0 or is_terminal:
        if is_terminal:
            if winning_move(board, AI_PIECE):
                return (None, math.inf)
            elif winning_move(board, PLAYER_PIECE):
                return (None, -math.inf)
            else:  # Game is over, no more valid moves
                return (None, 0)
        else:  # Depth is zero
            return (None, score_position(board, AI_PIECE))
    if maximizingPlayer:
        value = -math.inf
        best_col = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = [row[:] for row in board]
            drop_piece(b_copy, row, col, AI_PIECE)
            new_score = minimax(b_copy, depth-1, alpha, beta, False)[1]
            if new_score > value:
                value = new_score
                best_col = col
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return best_col, value

    else:  # Minimizing player
        value = math.inf
        best_col = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = [row[:] for row in board]
            drop_piece(b_copy, row, col, PLAYER_PIECE)
            new_score = minimax(b_copy, depth-1, alpha, beta, True)[1]
            if new_score < value:
                value = new_score
                best_col = col
            beta = min(beta, value)
            if alpha >= beta:
                break
        return best_col, value

def get_valid_locations(board):
    valid_locations = []
    for col in range(COLUMNS):
        if is_valid_location(board, col):
            valid_locations.append(col)
    return valid_locations

def play_game():
    board = create_board()
    game_over = False

    print("Choose game mode:")
    print("1. AI vs Human")
    print("2. AI vs AI")
    mode = int(input("Enter mode (1 or 2): "))

    turn = random.randint(0, 1)  # Randomly choose who goes first

    while not game_over:
        if mode == 1:
            if turn == 0:
                # Human turn
                print_board(board)
                try:
                    col = int(input("Player 'O', make your selection (0-6): "))
                except ValueError:
                    print("Invalid input. Please enter an integer between 0 and 6.")
                    continue
                if col < 0 or col >= COLUMNS:
                    print("Column out of bounds. Try again.")
                    continue
                if is_valid_location(board, col):
                    row = get_next_open_row(board, col)
                    drop_piece(board, row, col, PLAYER_PIECE)

                    winning_positions = winning_move(board, PLAYER_PIECE)
                    if winning_positions:
                        print_board(board, winning_positions)
                        print("Player 'O' WINS!")
                        game_over = True
                else:
                    print("Column full. Try a different column.")
                    continue
            else:
                # AI Turn
                col, minimax_score = minimax(board, 5, -math.inf, math.inf, True)
                if is_valid_location(board, col):
                    row = get_next_open_row(board, col)
                    drop_piece(board, row, col, AI_PIECE)

                    winning_positions = winning_move(board, AI_PIECE)
                    if winning_positions:
                        print_board(board, winning_positions)
                        print("Player 'X' WINS!")
                        game_over = True

            turn += 1
            turn = turn % 2

        elif mode == 2:
            # AI vs AI
            print_board(board)
            if turn == 0:
                # AI 1
                col, minimax_score = minimax(board, 5, -math.inf, math.inf, True)
                piece = PLAYER_PIECE
            else:
                # AI 2
                col, minimax_score = minimax(board, 5, -math.inf, math.inf, True)
                piece = AI_PIECE

            if is_valid_location(board, col):
                row = get_next_open_row(board, col)
                drop_piece(board, row, col, piece)

                winning_positions = winning_move(board, piece)
                if winning_positions:
                    print_board(board, winning_positions)
                    print(f"Player '{get_piece_symbol(piece)}' WINS!")
                    game_over = True

            turn += 1
            turn = turn % 2

        else:
            print("Invalid mode selected.")
            break

        if len(get_valid_locations(board)) == 0:
            print_board(board)
            print("Game is a draw.")
            game_over = True

if __name__ == "__main__":
    play_game()


Choose game mode:
1. AI vs Human
2. AI vs AI
0  1  2  3  4  5  6
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .

0  1  2  3  4  5  6
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  O  .  .  .

0  1  2  3  4  5  6
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  X  .  .  .
.  .  .  O  .  .  .

0  1  2  3  4  5  6
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  O  .  .  .
.  .  .  X  .  .  .
.  .  .  O  .  .  .

0  1  2  3  4  5  6
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  O  .  .  .
.  .  .  X  .  .  .
.  .  X  O  .  .  .

0  1  2  3  4  5  6
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  O  .  .  .
.  .  .  O  .  .  .
.  .  .  X  .  .  .
.  .  X  O  .  .  .

0  1  2  3  4  5  6
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  O  .  .  .
.  .  .  O  .  .  .
.  .  X  

**Explanation:**

- **Board Representation:** The board is a 2D list with dimensions 6 rows by 7 columns.
- **Game Logic Functions:**
  - `create_board()` initializes the game board.
  - `drop_piece()` places a piece on the board.
  - `is_valid_location()` checks if a column can accept another piece.
  - `get_next_open_row()` finds the next available row in a column.
  - `winning_move()` checks if the last move resulted in a win.
- **AI Algorithm:**
  - The AI uses the Minimax algorithm with alpha-beta pruning to a depth of 5.
  - The `minimax()` function evaluates the board recursively.
  - `score_position()` provides a heuristic evaluation of the board.
  - `evaluate_window()` scores windows of four cells.
- **Game Modes:**
  - **AI vs Human:** The human plays against the AI. The human is prompted to enter a column number.
  - **AI vs AI:** Two AI agents play against each other.
- **Gameplay:**
  - The game randomly selects which player goes first.
  - The game continues until a player wins or the board is full (draw).

**How to Use the Code:**

1. Run the script.
2. When prompted, select game mode:
   - Enter `1` for AI vs Human.
   - Enter `2` for AI vs AI.
3. If playing AI vs Human and it's your turn, input a column number between 0 and 6 to make your move.
4. The game will display the board after each move.